In [12]:
from fase import seal

계산을 시작하기 전 scheme의 context를 설정해야함.  
설정에 필요한 parameter들은 `EncryptionParameters`로 설정가능.  
주요한 parameter는 다음과 같음 :
- poly_modulus_degree : Cyclotomic polynomial ring의 크기를 결정. 
- coeff_modulus ([ciphertext] coefficient modulus)
- plain_modulus (plaintext modulus; only for the BFV scheme).

In [13]:
parms = seal.EncryptionParameters(seal.scheme_type.bfv)

## 0. Noise budget
FHE scheme들은 inverse 계산을 불가능하게 하기 위해 계산에 noise를 추가함.  
그런데 연산이 진행되면 noise가 점점 증가함 (특히 ciphertext * ciphertext) 그래서 noise budget이라는 개념이 사용됨.  
문제의 Multilication depth를 미리 확인하고 적절한 noise budget을 설정해야함. (무조건 키울수는 없음 -- 왜? )

## 1. Parameters

### 1. 1. Polynomial modulus (poly_modulus_degree)

Polynomical modulus의 degree를 뜻하며, 2^x 형태임. 일반적으로 2^10, ^11, .. ^15정도를 사용함. 

In [14]:
p = 4096
parms.set_poly_modulus_degree(p)

### 1. 2. (Ciphertext) coefficient modulus
서로 다른 prime number의 곱인 어떤 수이며, 각각의 prime number는 최대 60bit임.  
coeff_modulus가 클 수록 noise budget이 커짐.
coeff_modulus의 최대치는 poly_modulus_degree로 결정 됨.  


        +----------------------------------------------------+
        | poly_modulus_degree | max coeff_modulus bit-length |
        +---------------------+------------------------------+
        | 1024                | 27                           |
        | 2048                | 54                           |
        | 4096                | 109                          |
        | 8192                | 218                          |
        | 16384               | 438                          |
        | 32768               | 881                          |
        +---------------------+------------------------------+

#### 1. 2.1 Convenience function CoeffModulus.BFVDefault()
... returns a generally good choice for a given polynomial modulus.

In [15]:
parms.set_coeff_modulus(seal.CoeffModulus.BFVDefault(p))

## 참고) poly_modulus_degree에 따른 max coeff_modulus 계산해주는 함수
seal.CoeffModulus.MaxBitCount(p)

109

### 1. 3. Plaintext modulus (Plain_modulus)
plaintext modulus는 BFV에서 사용됨. 모든 정수가 가능하지만 prime number면 더 좋음.  
plaintext modulus는 plaintext의 최대 크기를 결정하고, 거기에 따라 곱하기때 noise consumption이 결정 됨. 따라서 Plaintext modulus를 불필요하고 크게 잡지 않는 것이 좋음.    

noise budget = log2(coeff mod/plain mod) (bits)


** 참고로 CKKS에서 noise budget consumption을 결정하는 파라미터는 ??? 임. (logp of HEAAN)

In [16]:
parms.set_plain_modulus(1024) # noise budget = log2(4096/1024)  = log2(4) = 2  : 뭔가 이상한데../ 

(곱하기때) noise consumption = log2(plain_modulus) + (other terms) 이므로, 지금 상태론 곱하기 불가능! 

In [17]:
context = seal.SEALContext(parms) # parameters are internally validated. 

### 2. 1 Key generation
SEAL의 scheme들은 public key - sceret key 쌍으로 동작함. Secret key를 가진 사람만 decrypt 가능.

In [18]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()

# 근데 key를 만들때 쓰는 random number는 seed와 무관하겠지?

### 2.2 Agents

encryption / decryption / ciphertext 계산을 위한 agent가 있음. 

encryptor는 public key를 사용하고,   
decryptor는 secret key를 사용함.

In [19]:
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

## 3. Performing computation
### Encoding to ptext

Batching 없는 단순한 modulo 연산 예제 -> 별도의 encoding 없이 integer를 hex로 바꾸어 넣어주면 됨. 왠지 모름... 

일반적으로는 encoder.encode()로 encoding함. 
** BFV와 CKKS의 encoder는 전혀 다른 방식으로 작동함. 

In [20]:
def int_to_hex_string(vv):
    return f"{vv:x}"

In [21]:
vv = 3
x_plain = seal.Plaintext(int_to_hex_string(vv))
print(x_plain.to_string()) # supposed to be the same as int_to_hex_string(vv)

3


#### Encrypting to ctext

In [22]:
def check_budget(ctxt):
    print(f"The size of encrypted text:, {ctxt.size()} bits") 
    print(f"Noise budget: {decryptor.invariant_noise_budget(ctxt)} bits")

In [25]:
x_enc = encryptor.encrypt(x_plain)
check_budget(x_enc)
# noise budget 한참 남음. 

The size of encrypted text:, 2 bits
Noise budget: 55 bits


In [27]:
x_dec = decryptor.decrypt(x_enc)
print(x_dec.to_string(), "==", int_to_hex_string(vv))
print("Seems I can encrypt and decrypt correctly!")

3 == 3
Seems I can encrypt and decrypt correctly!


### 곱하기 depth 줄이기 
계산을 짤때 곱하기 수를 줄이는게 좋음. x^4 + 2x^2 + 1은 곱하기가 최대 4번 되지만, (x^2+1) * (x^2+1)은 곱하기가 최대 3번임. 

우리는 $$4x^4 + 8x^3 + 8x^2 + 8x + 4 = 4(x + 1)^2 \times (x^2 + 1)$$ 계산할거임.

In [28]:
print("Compute x^2+1")
x_sq = evaluator.square(x_enc)
x_sq_plus_one = evaluator.add_plain(x_sq, seal.Plaintext("1")) # 1은 hex로도 1

check_budget(x_sq_plus_one)

Compute x^2+1
The size of encrypted text:, 3 bits
Noise budget: 33 bits


budget이 좀 줄어들었음. 

In [31]:
decrypted = decryptor.decrypt(x_sq_plus_one).to_string()
print(decrypted)

# are they the same?
print(vv**2+1, int(decrypted, base=16))

A
10 10


***plain_modulus를 1024로 설정했기 때문에 결과값이 정답//1024로 표기됨.***

In [32]:
# compute (x+1)^2
x_plus_1_sq = evaluator.add_plain(x_enc, seal.Plaintext('1'))
evaluator.square_inplace(x_plus_1_sq) # in_place 버전도 있음
check_budget(x_plus_1_sq)

The size of encrypted text:, 3 bits
Noise budget: 33 bits


In [33]:
# multiply (x^2+1) * (x+1)^2 * 4
result = evaluator.multiply(x_sq_plus_one, x_plus_1_sq)
evaluator.multiply_plain_inplace(result, seal.Plaintext('4'))
check_budget(result)

The size of encrypted text:, 5 bits
Noise budget: 3 bits


budget 거의 다 씀. 휴.. 

In [34]:
dec_result = decryptor.decrypt(result)
print(4*vv**4+8*vv**3+8*vv**2+8*vv+4, int(dec_result.to_string(), base=16))
# 같음?

640 640


이상하다.. 예제에선 budget다 써서 계산이 틀려야한다는데... 왜 맞을까.

## Budget 관리: Relinearization
계산 중간중간 (곱하기 할때마다) relinearization을 해주면 ctxt 크기가 작아지고, 계산이 빨라지고, budget consumption이 줄어듬.   
* Relinearization은 3bit polynomial을 2bit로 줄이는 것만 가능.
* 'Relinearization key'가 따로 필요함. 한번에 하나씩
* BFV와 CKKS에서 비슷하게 사용됨. 

In [204]:
vv = 10
x_plain = seal.Plaintext(int_to_hex_string(vv))
x_enc = encryptor.encrypt(x_plain)

In [205]:
relin_keys = keygen.create_relin_keys()

# 아까랑 같은데 군데군데 relin을 섞어줌.
x_sq = evaluator.square(x_enc)
evaluator.relinearize_inplace(x_sq, relin_keys)
x_sq_plus_one = evaluator.add_plain(x_sq, seal.Plaintext("1"))
check_budget(x_sq_plus_one)

x_plus_1_sq = evaluator.add_plain(x_enc, seal.Plaintext('1'))
evaluator.square_inplace(x_plus_1_sq)
evaluator.relinearize_inplace(x_plus_1_sq, relin_keys)

check_budget(x_plus_1_sq)

result = evaluator.multiply(x_sq_plus_one, x_plus_1_sq)
evaluator.relinearize_inplace(result, relin_keys)
evaluator.multiply_plain_inplace(result, seal.Plaintext('4'))
check_budget(result)


dec_result = decryptor.decrypt(result)
print("결과확인")
print(4*vv**4+8*vv**3+8*vv**2+8*vv+4, int(dec_result.to_string(), base=16))

The size of encrypted text:, 2 bits
Noise budget: 33 bits
The size of encrypted text:, 2 bits
Noise budget: 33 bits
The size of encrypted text:, 2 bits
Noise budget: 10 bits
결과확인
48884 756


In [207]:
48884 % 1024

756

1. ctxt 크기가 계속 2bit임! 
2. budget이 최종적으로 10bit 남음. 이전엔 3bit이었는데. 
3. 결과는 % 1024 값으로 나옴. -- 그럼 원래 값을 어떻게 알아...?? 